## Preprocessing – Heart Disease Dataset

Ce notebook est dédié à la phase de preprocessing du jeu de données *Heart Disease*.
Il comprend le nettoyage des données, la gestion des valeurs manquantes, l’encodage
des variables catégorielles, la mise à l’échelle des variables numériques ainsi que
la mise en place d’un pipeline reproductible en vue de la modélisation.


In [2]:
from pathlib import Path
import pandas as pd

DATA_PATH = Path("..") / "dataset.csv"
df = pd.read_csv(DATA_PATH)

df.head()


,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


## Séparation des variables explicatives et de la cible

Les données sont séparées en :
- `X` : les variables explicatives
- `y` : la variable cible (`num`)


In [3]:
X = df.drop(columns=["num"])
y = df["num"]

X.shape, y.shape

((920, 15), (920,))

## Mise en place du pipeline de preprocessing

Un pipeline est mis en place afin de gérer :
- l’imputation des valeurs manquantes
- l’encodage des variables catégorielles
- la standardisation des variables numériques

Cette approche permet d’éviter toute fuite de données et d’assurer la reproductibilité.


In [4]:
num_features = X.select_dtypes(include=["int64", "float64"]).columns
cat_features = X.select_dtypes(include=["object"]).columns

num_features, cat_features

(Index(['id', 'age', 'trestbps', 'chol', 'thalch', 'oldpeak', 'ca'], dtype='object'),
 Index(['sex', 'dataset', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal'], dtype='object'))

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


In [6]:
numeric_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])


In [7]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_pipeline, num_features),
        ("cat", categorical_pipeline, cat_features)
    ]
)


## Séparation du jeu de données

Les données sont divisées en un jeu d’entraînement et un jeu de test.
La stratification est utilisée afin de conserver la distribution de la variable cible.


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape


((736, 15), (184, 15))

In [9]:
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

X_train_processed.shape, X_test_processed.shape


((736, 30), (184, 30))

## Conclusion du preprocessing

Les données ont été préparées à l’aide de pipelines distincts pour les variables numériques et catégorielles.  
Cette approche garantit une transformation cohérente des données et évite toute fuite d’information vers le jeu de test.  
Le pipeline de preprocessing sera intégré directement aux modèles lors de la phase de modélisation.
